In [ ]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error
from datetime import datetime

In [ ]:
subset = 'FD001' # subset data yang ingin digunakan
n_xgb = '10' # jumlah fitur terbaik yang ingin dipilih
now = datetime.now()
formatted = now.strftime("%d-%m-%Y %H:%M")

# Data Train Process

In [ ]:
df = pd.read_csv('../CMAPSSData/train_' + subset + '.csv')
df

In [ ]:
def piecewise_linear_rul(df, rul_max=128):
    def compute_rul(cycles):
        max_cycle = cycles.max()
        rul = max_cycle - cycles
        return np.where(rul > rul_max, rul_max, rul)

    df['RUL'] = df.groupby('unit_number')['cycles'].transform(compute_rul)
    return df

piecewise_linear_rul(df)

In [ ]:
X_xgb = df.drop(columns=['unit_number', 'cycles', 'RUL'])
y_xgb = df['RUL'] + 1

print(np.isnan(X_xgb).any()) 
print(np.isnan(y_xgb).any()) 

In [ ]:
xgb_model = XGBRegressor(
    booster='gbtree',
    objective='reg:gamma',
    gamma=0.1,   
    reg_lambda=3,
    subsample=0.7,
    learning_rate=0.05,
    max_depth=5,
    min_child_weight=7, 
    n_estimators=250
)
xgb_model.fit(X_xgb, y_xgb)

In [ ]:
features = [col for col in df.columns if col not in ['unit_number', 'cycles', 'RUL']]

In [ ]:
sorted_indices = np.argsort(xgb_model.feature_importances_)[::-1]
top_n = int(n_xgb)  # Jumlah fitur terbaik yang diinginkan

# Ambil fitur terbaik
selected_features = np.array(features)[sorted_indices[:top_n]]

print("Fitur yang terpilih:", selected_features)


In [ ]:
features = selected_features.tolist()

In [ ]:
df_selected = df[['unit_number', 'cycles'] + selected_features.tolist() + ['RUL']].copy()
df_selected

In [ ]:
# Normalisasi data
mean = df_selected[selected_features].mean()
std = df_selected[selected_features].std()

# Terapkan normalisasi
df_selected[selected_features] = (df_selected[selected_features] - mean) / std
df_selected

In [ ]:
# Sliding window
window_size=39
def create_sliding_window(data, window_size=50, stride=1):
    windows = []
    labels = []
    for unit in data['unit_number'].unique():
        unit_data = data[data['unit_number'] == unit]
        for i in range(0, len(unit_data) - window_size + 1, stride):
            windows.append(unit_data.iloc[i:i+window_size][features].values)
            labels.append(unit_data.iloc[i+window_size-1]['RUL'])
    return np.array(windows), np.array(labels)

X, y = create_sliding_window(df_selected, window_size=window_size, stride=1)


print(np.isnan(X).any())  # Periksa NaN pada X
print(np.isinf(X).any())  # Periksa Infinity pada X
print(np.isnan(y).any())  # Periksa NaN pada y
print(np.isinf(y).any())  # Periksa Infinity pada y

print(f"Shape of input data: {X.shape}")
print(f"Shape of labels: {y.shape}")

In [ ]:
from sklearn.model_selection import train_test_split

# Pisahkan data train menjadi train set dan validation set
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=237)

print(f"Shape X_train: {X_train.shape}")
print(f"Shape y_train: {y_train.shape}")
print(f"Shape X_val: {X_val.shape}")
print(f"Shape y_val: {y_val.shape}")


# Data Test Procces

In [ ]:
df_test = pd.read_csv('../CMAPSSData/test_' + subset + '.csv')
df_test

In [ ]:
print(np.isnan(df_test).any())

In [ ]:
df_test_selected = df_test[['unit_number', 'cycles'] + selected_features.tolist()].copy()
df_test_selected

In [ ]:
# Terapkan normalisasi
df_test_selected[selected_features] = (df_test_selected[selected_features] - mean) / std
df_test_selected

In [ ]:
def pad_window(data, window_size, features):
    if len(data) < window_size:
        padding = np.zeros((window_size - len(data), len(features)))  # Padding dengan nol
        padded_data = np.vstack((padding, data))
    else:
        padded_data = data[-window_size:]  # Ambil window terakhir
    return padded_data

X_test = []
for unit in df_test_selected['unit_number'].unique():
    unit_data = df_test_selected[df_test_selected['unit_number'] == unit][features].values
    test_window = pad_window(unit_data, window_size=window_size, features=features)
    X_test.append(test_window)

X_test = np.array(X_test)
X_test.shape

# RUL Process

In [ ]:
y_actual = pd.read_csv('../CMAPSSData/RUL_' + subset + '.csv')
y_actual

# Model Building

In [ ]:
def create_lstm_model(input_shape, seed):
    initializer = tf.keras.initializers.GlorotUniform(seed=seed)
    model = Sequential()
    model.add(Masking(mask_value=0.0, input_shape=input_shape))
    model.add(LSTM(32, activation='relu', return_sequences=True))
    model.add(LSTM(16, activation='relu', kernel_initializer=initializer))
    model.add(Dense(1, kernel_initializer=initializer))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='mean_squared_error')
    return model

# Model Training

In [ ]:
batch_size = 32
epochs = 100
n_models = 1 # Jumlah model dalam ensemble. 1 = Model tunggal, >1 = Ensemble (15 atau dst)
input_shape = (window_size, len(features))
ensemble_models = []
ensemble_histories = []

early_stopping = EarlyStopping(monitor='val_loss', patience=3, min_delta=0.001,
                                start_from_epoch=85, restore_best_weights=True)

for i in range(n_models):
    print(f"Training model {i+1}/{n_models}")
    model = create_lstm_model(input_shape, seed=237 + i)
    history = model.fit(X_train, y_train,
                        validation_data=(X_val, y_val),
                        epochs=epochs,
                        batch_size=batch_size,
                        callbacks=[early_stopping]
                        )
    ensemble_models.append(model)
    ensemble_histories.append(history)
    model.save(f'models_{n_models}_{subset}_{n_xgb}xgboost_{formatted}/models/ensemble_model_{i}.h5')  # Simpan model individu


In [ ]:
import matplotlib.pyplot as plt
import math

# Jumlah model dalam ensemble
num_models = len(ensemble_histories)

# Tentukan jumlah baris dan kolom (2 plot per baris)
cols = 5
rows = math.ceil(num_models / cols)  # Hitung jumlah baris yang dibutuhkan

# Buat subplot untuk setiap model
fig, axes = plt.subplots(rows, cols, figsize=(15, rows * 5))  # Ukuran subplot
axes = axes.flatten()  # Flatten array axes untuk akses mudah

# Plot loss untuk setiap model
for i, history in enumerate(ensemble_histories):
    ax = axes[i]
    ax.plot(history.history['loss'], label='Training Loss', color='blue')
    ax.plot(history.history['val_loss'], label='Validation Loss', color='orange', linestyle='--')
    ax.set_title(f'Model {i+1}', fontsize=12)
    ax.set_xlabel('Epochs')
    ax.set_ylabel('Loss')
    ax.legend(loc='upper right')
    ax.grid(True)

# Hapus axes kosong jika jumlah model kurang dari jumlah subplot
for j in range(len(axes)):
    if j >= num_models:
        fig.delaxes(axes[j])

# Tambahkan judul utama dan tata letak
fig.suptitle('Training and Validation Loss for Ensemble Models', fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.96])  # Sesuaikan ruang untuk judul utama
plt.show()


In [ ]:
class WeightedEnsemble(tf.keras.Model):
    def __init__(self, models, weights):
        super(WeightedEnsemble, self).__init__()
        self.models = models
        self.model_weights = tf.constant(weights, dtype=tf.float32)  # Ganti nama dari self.weights

    def call(self, inputs):
        outputs = tf.stack([model(inputs, training=False) for model in self.models], axis=0)
        return tf.tensordot(self.model_weights, outputs, axes=1)
    
# Hitung bobot berdasarkan val_loss
weights = []
valid_models = []
valid_histories = []
for model, history in zip(ensemble_models, ensemble_histories):
    final_val_loss = history.history['val_loss'][-1]
    if not np.isnan(final_val_loss):
        weights.append(1 / final_val_loss)
        valid_models.append(model)
        valid_histories.append(history)

normalized_weights = np.array(weights) / np.sum(weights)
ensemble_combined_model = WeightedEnsemble(valid_models, normalized_weights)



# Prediction and Evaluation

In [ ]:
def ensemble_predict_probabilistic(models, X, histories):
    valid_models = []
    weights = []
    for model, history in zip(models, histories):
        final_val_loss = history.history['val_loss'][-1]
        if not np.isnan(final_val_loss):
            valid_models.append(model)
            weights.append(1 / final_val_loss)

    weights = np.array(weights)
    normalized_weights = weights / np.sum(weights)
    predictions = [model.predict(X) for model in valid_models]
    weighted_predictions = np.average(predictions, axis=0, weights=normalized_weights)
    std_predictions = np.sqrt(np.average((predictions - weighted_predictions) ** 2,
                                         axis=0, weights=normalized_weights))
    return weighted_predictions, std_predictions

def calculate_rmse(y_actual, y_pred):
    return np.sqrt(mean_squared_error(y_actual, y_pred))

def calculate_picp(y_actual, mean_pred, delta):
    lower_bound = mean_pred - delta
    upper_bound = mean_pred + delta
    within_interval = np.logical_and(y_actual >= lower_bound, y_actual <= upper_bound)
    return np.mean(within_interval)

def calculate_nmpiw(y_actual, delta):
    interval_width = 2 * delta
    return np.mean(interval_width) / (np.max(y_actual) - np.min(y_actual))

mean_pred, std_pred = ensemble_predict_probabilistic(ensemble_models, X_test, ensemble_histories)
z = 1.96
delta = z * std_pred

rmse = calculate_rmse(y_actual, mean_pred)
picp = calculate_picp(y_actual, mean_pred, delta)
nmpiw = calculate_nmpiw(y_actual, delta)

print(f"RMSE: {rmse}")
print(f"PICP: {picp}")
print(f"NMPIW: {nmpiw}")

In [ ]:
# Simpan model gabungan

ensemble_combined_model.save(f'models_{n_models}_{subset}_{n_xgb}xgboost_{formatted}/{round(rmse, 4)}_{round(picp, 4)}_{round(nmpiw, 4)}_ensemble_model_combined.h5')
